In [4]:
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
import os
warnings.filterwarnings('ignore')

In [8]:
# Genres
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
    
    # Write the transcription to a file
    with open("transcription_genres.txt", "w") as file:
        file.write(transcription)
    
    return transcription

# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(share=False)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [16]:
#genre analysis with OpenAI
#preprocess txt file

with open('transcription_genres.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [17]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine three movie genres from the following list: Action, Adventure, Animation, Comedy, Crime, Documentary, Drama, Family, Fantasy, History, Horror, Music, Mystery, Romance, Science Fiction, TV Movie, Thriller, War, and Western based on the provided text."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"Preference: {preference}")

analyze_document('transcription_genres.txt')


Preference: Action, Adventure, Comedy


Determine three types of movies genres preferred by the provided text ranked in order of preference.

Preference: 1. "E.T. the Extra-Terrestrial" - This iconic Steven Spielberg film notably features a scene with the alien character E.T. consuming Reese's Pieces (a form of peanut butter candy) and includes many bicycle scenes.

2. "Paul" - This comedy film follows two comic book enthusiasts who encounter an alien near Area 51 - while it doesn't directly feature bicycles or peanut butter, it has a similar comedic sensibility that may appeal to someone with those interests.

3. "Race to Witch Mountain" - This adventure film features two alien siblings using their powers on earth. While it doesn't explicitly involve peanut butter or bicycles, it has the look and feel of movies that would fit those criteria.

Determine three movies based on the preferences in the provided text.



In [ ]:
#Change Gradio Interface (gradio theme builder)

gr.themes.builder()